# Requirements

In [1]:
import os
import json
import pandas as pd
import numpy as np
import json
import requests

import re
from itertools import islice # to iterate through dicts

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

In [2]:
LAGT = pd.read_json("../data/large_files/LAGT_grecy_20240116.json")

In [3]:
LAGT.head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source
0,ggm0001,ggm0001.ggm001,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr,"[[Χρής, γιγνώσκω, γῆ, περίμετρος], [στάδιος, μ...",None
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",None
3,stoa0033a,stoa0033a.tlg028,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr,"[[θεῖος, δαιμόνιον, χρῆμα, Ἀλέξανδρος, φιλοσοφ...",None
4,stoa0033a,stoa0033a.tlg043,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr,"[[ἔμφυτος, πνεῦμα, διαμονή, αὔξησις], [Ὁρῶμεν,...",None
5,stoa0121,stoa0121.stoa001,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr,"[[Ῥωμαϊκός, βασιλεία, προοίμιον, οὐδείς, γίγνο...",None


In [4]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/


In [5]:
file_data = json.load(open("../../ServiceAccountsKey.json"))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
  ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

AGT_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1iVta_FuEDgUM_Lf_yByrdbbXNoVH_dnVZs6QRyYv1NM/edit?usp=sharing")
# AGT_metadata = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hEUnL3E07F-EnE3wYnk1V91aXfPDrcnhFHKjD-04CM0/edit?usp=sharing")

# Extract tlg metadata from cltk github

In [4]:
tlg_authordate = requests.get("https://raw.githubusercontent.com/cltk/cltk/master/src/cltk/corpora/grc/tlg/author_date.json").json()

In [5]:
### write it out for future usage
#s.write_file("SDAM_data/OGL/tlg_authordate.json", tlg_authordate)

In [8]:
def parse_date_str(date_str):
  not_before, not_after = None, None
  if ("B.C." in date_str) & ("A.D." in date_str):
    match = re.match("([p|a]\.\s)?(\d+)?\s(B\.C\.)(\?)?(-|/)([p|a]\.\s)?(A\.D\.\s)(\d+)(\?)?", date_str)
    try:
      not_before = int(match.groups()[1]) * -100
      not_after = int(match.groups()[-2]) * 100
      if ("a. " in date_str) & ("p. " in date_str):
        not_before += 100
        not_after -= 100
    except:
      pass
  elif "B.C." in date_str:
    match = re.match("([p|a]\.\s)?(\d+)(\?)?(-|/)?(\d+)?(\?)?\s(B\.C\.)(\?)?$", date_str)
    try:
      if match.groups()[0] == "p. ":
        if match.groups()[4] != None:
          not_before = (int(match.groups()[4]) - 1)   * -100
        else:
          not_before = (int(match.groups()[1]) - 1)   * -100
      elif match.groups()[0] == "a. ":
        not_after = (int(match.groups()[1])  * -100) - 1
      else:
        not_before = int(match.groups()[1]) * -100
        if match.groups()[4] != None:
          not_after = (int(match.groups()[4]) * -100) + 99
        else:
          not_after = (int(match.groups()[1]) * -100) + 99
    except:
      pass
  else:
    match = re.match("([p|a]\.\s)?(A\.D\.\s)(\d+)(\?)?(-|/)?(\d+)?(\?)?$", date_str)
    try:
      if match.groups()[0] == "p. ":
        if match.groups()[5] != None:
          not_before = int(match.groups()[5])   * 100 +1
        else:
          not_before = int(match.groups()[2])   * 100 +1
      elif match.groups()[0] == "a. ":
        not_after = (int(match.groups()[2]) - 1)  * 100
      else:
        not_before = (int(match.groups()[2]) * 100) - 99
        if match.groups()[4] != None:
          not_after = (int(match.groups()[5]) * 100) 
        else:
          not_after = (int(match.groups()[2]) * 100)
    except:
      pass
  if not_after == 0:
    not_after = -1
  if not_before == 0:
      not_before = 1
  date_dict = {"not_before" : not_before, "not_after" : not_after}
  if "?" in date_str:
    date_dict["date_uncertain"] = True
  else:
    date_dict["date_uncertain"] = False
  return date_dict 

In [9]:
dates_dict = {}
for date_str in tlg_authordate.keys():
  dates_dict[date_str] = parse_date_str(date_str)
dates_dict

{'1 B.C.': {'not_before': -100, 'not_after': -1, 'date_uncertain': False},
 '1 B.C.-A.D. 1': {'not_before': -100,
  'not_after': 100,
  'date_uncertain': False},
 '1 B.C.-A.D. 1?': {'not_before': -100,
  'not_after': 100,
  'date_uncertain': True},
 '1 B.C./A.D. 1': {'not_before': -100,
  'not_after': 100,
  'date_uncertain': False},
 '1 B.C./A.D. 1?': {'not_before': -100,
  'not_after': 100,
  'date_uncertain': True},
 '1 B.C./A.D. 4?': {'not_before': -100,
  'not_after': 400,
  'date_uncertain': True},
 '1 B.C.?': {'not_before': -100, 'not_after': -1, 'date_uncertain': True},
 '2 B.C.': {'not_before': -200, 'not_after': -101, 'date_uncertain': False},
 '2 B.C.-A.D. 4': {'not_before': -200,
  'not_after': 400,
  'date_uncertain': False},
 '2 B.C./A.D. 2': {'not_before': -200,
  'not_after': 200,
  'date_uncertain': False},
 '2 B.C./A.D. 2?': {'not_before': -200,
  'not_after': 200,
  'date_uncertain': True},
 '2 B.C./A.D. 3': {'not_before': -200,
  'not_after': 300,
  'date_uncertain'

### Load LAGT

In [20]:
LAGT = s.read_file("SDAM_data/AGT/AGT_raw_20240104.json", "df")

In [47]:
LAGT["author_id"] = LAGT.apply(lambda row: row["filename"][:7], axis=1)

In [48]:
LAGT.head(10)

,filename,author,title,string,wordcount,source,author_id,tlg_date,not_before,not_after,date_uncertain
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr,ggm0001,None,NaN,NaN,None
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias,FROM HEBREW I. 1 The vision of Isaiah the son ...,40168,1Kgr,heb0001,None,NaN,NaN,None
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,ogl0001,None,NaN,NaN,None
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr,stoa003,None,NaN,NaN,None
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr,stoa003,None,NaN,NaN,None
5,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr,stoa012,None,NaN,NaN,None
6,stoa0146d.stoa001.opp-grc1.xml,Hegemonius,Acta Archelai,Μανιχαῖος ἀπόστολος Ἰησοῦ Χριστοῦ καὶ οἱ σὺν ἐ...,5166,1Kgr,stoa014,None,NaN,NaN,None
7,stoa0146d.stoa001.opp-lat1.xml,Hegemonius,Acta Archelai,Thesaurus verus sive disputatio habita in Carc...,42159,1Kgr,stoa014,None,NaN,NaN,None
8,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,tlg0005,4-3 B.C.,-400.0,-201.0,False
9,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,tlg0006,5 B.C.,-500.0,-401.0,False


### Map TLG dates on LAGT

In [49]:
tlg_authordate_reversed = {}
for key, val in tlg_authordate.items():
  for id in val:
    tlg_authordate_reversed["tlg" + id] = key 

In [50]:
def get_date(author_id):
  try:
    return tlg_authordate_reversed[author_id]
  except:
    return None
LAGT["tlg_date"] = LAGT["author_id"].apply(get_date)

In [51]:
for col in ["not_before", "not_after", "date_uncertain"]:
  LAGT[col] = LAGT["tlg_date"].apply(lambda x: dates_dict[x][col] if x is not None else None)

In [52]:
LAGT.head(10)

,filename,author,title,string,wordcount,source,author_id,tlg_date,not_before,not_after,date_uncertain
0,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr,ggm0001,None,NaN,NaN,None
1,heb0001.heb010.1st1K-eng1.xml,None,Isaias,FROM HEBREW I. 1 The vision of Isaiah the son ...,40168,1Kgr,heb0001,None,NaN,NaN,None
2,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,ogl0001,None,NaN,NaN,None
3,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr,stoa003,None,NaN,NaN,None
4,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr,stoa003,None,NaN,NaN,None
5,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr,stoa012,None,NaN,NaN,None
6,stoa0146d.stoa001.opp-grc1.xml,Hegemonius,Acta Archelai,Μανιχαῖος ἀπόστολος Ἰησοῦ Χριστοῦ καὶ οἱ σὺν ἐ...,5166,1Kgr,stoa014,None,NaN,NaN,None
7,stoa0146d.stoa001.opp-lat1.xml,Hegemonius,Acta Archelai,Thesaurus verus sive disputatio habita in Carc...,42159,1Kgr,stoa014,None,NaN,NaN,None
8,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,tlg0005,4-3 B.C.,-400.0,-201.0,False
9,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,tlg0006,5 B.C.,-500.0,-401.0,False


### Clean some metadata manually (Homeric hymns and Greek New Testament)

In [54]:
LAGT["not_before_man"] = LAGT["not_before"]
LAGT["not_after_man"] = LAGT["not_after"]

In [57]:
LAGT["doc_id"] = LAGT["filename"].apply(lambda x: x.rpartition(".")[0].rpartition(".")[0])

In [58]:
## updated dates for the longest homeric hymns, based on:
### Faulkner, Andrew, ‘Introduction. Modern Scholarship on the Homeric Hymns: Foundational Issues’, in The Homeric Hymns: Interpretative Essays, ed. by Andrew Faulkner (Oxford: Oxford University Press, 2011), pp. 1–25
LAGT.loc[LAGT["doc_id"].str.startswith("tlg0013"), "author"] = "Homeric hymn"


LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg001", "author_id"] = "tlg0013dyon"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg001", "not_before_man"] = -700
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg001", "not_after_man"] = -601


LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg002", "author_id"] = "tlg0013deme"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg002", "not_before_man"] = -700
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg002", "not_after_man"] = -501


LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg003", "author_id"] = "tlg0013apol"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg003", "not_before_man"] = -700
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg003", "not_after_man"] = -501


LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg004", "author_id"] = "tlg0013herm" # "Ultimately, a late-sixth-century date for Hermes seems most attractive, but the Hymn could also belong to the ﬁrst half of the ﬁfth century."
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg004", "not_before_man"] = -600
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg004", "not_after_man"] = -401

LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg005", "author_id"] = "tlg0013aphr"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg005", "not_before_man"] = -700
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg005", "not_after_man"] = -501


LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg007", "author_id"] = "tlg0013dyo2"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg007", "not_before_man"] = -600
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg007", "not_after_man"] = -501

LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg019", "author_id"] = "tlg0013pan"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg019", "not_before_man"] = -600
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg019", "not_after_man"] = -501

### drop all remaining
#LAGT = LAGT[LAGT["author_id"] != "tlg0013"]

In [59]:
paul = ["tlg0031.tlg006", "tlg0031.tlg007", "tlg0031.tlg008", "tlg0031.tlg009", "tlg0031.tlg011", "tlg0031.tlg013", "tlg0031.tlg018"]
pauline = ["tlg0031.tlg010", "tlg0031.tlg012", "tlg0031.tlg014", "tlg0031.tlg015", "tlg0031.tlg016", "tlg0031.tlg017", ]
john = ["tlg0031.tlg004", "tlg0031.tlg023", "tlg0031.tlg024", "tlg0031.tlg025"]
luke = ["tlg0031.tlg003", "tlg0031.tlg005"]

for work in paul:
  LAGT.loc[LAGT["doc_id"]==work, "author_id"] = "tlg0031paul"
  LAGT.loc[LAGT["doc_id"]==work, "author"] = "Paul of Tarsus"

for work in pauline:
  LAGT.loc[LAGT["doc_id"]==work, "author_id"] = "tlg0031pspa"
  LAGT.loc[LAGT["doc_id"]==work, "author"] = "Pauline literature"

for work in john:
  LAGT.loc[LAGT["doc_id"]==work, "author_id"] = "tlg0031john"
  LAGT.loc[LAGT["doc_id"]==work, "author"] = "Johnannine literature"

for work in luke:
  LAGT.loc[LAGT["doc_id"]==work, "author_id"] = "tlg0031luke"
  LAGT.loc[LAGT["doc_id"]==work, "author"] = "Luke (the evangelist)"

In [60]:
import string
for letter, index in zip(list(string.ascii_lowercase)[:14], LAGT[LAGT["author_id"]=="tlg0031"].index.tolist()):
  LAGT.at[index, "author_id"] = "tlg0031" + letter

# Aggregate data for authors

In [61]:
authors_titles = LAGT.groupby(["author_id"])["title"].agg(list).reset_index(name="titles")
authors_titles_dict = authors_titles.set_index('author_id')['titles'].to_dict()

authors_sources = LAGT.groupby(["author_id"])["source"].agg(list).reset_index(name="sources")
authors_sources_dict = authors_sources.set_index('author_id')['sources'].to_dict()

authors_wordcounts_dict = LAGT.groupby(["author_id"]).sum()['wordcount'].to_dict()

In [75]:
authors_df = LAGT.drop_duplicates(subset="author_id").drop(columns=["filename", "title", "string", "wordcount", "source", "doc_id"])
authors_df["titles"] = authors_df["author_id"].map(authors_titles_dict)
authors_df["sources"] = authors_df["author_id"].map(authors_sources_dict)
authors_df["wordcounts"] = authors_df["author_id"].map(authors_wordcounts_dict)

In [76]:
authors_df.head(10)

,author,author_id,tlg_date,not_before,not_after,date_uncertain,not_before_man,not_after_man,titles,sources,wordcounts
0,Anonymous,ggm0001,None,NaN,NaN,None,NaN,NaN,[Anametresis Pontou],[1Kgr],382
1,None,heb0001,None,NaN,NaN,None,NaN,NaN,[Isaias],[1Kgr],40168
2,Pinytus,ogl0001,None,NaN,NaN,None,NaN,NaN,[De Epistola Pinyti ad Dionysium],[1Kgr],180
3,pseudo-Aristotle,stoa003,None,NaN,NaN,None,NaN,NaN,"[De mundo, De spiritu]","[1Kgr, 1Kgr]",9753
5,Eutropius,stoa012,None,NaN,NaN,None,NaN,NaN,[Breviarium historiae romanae],[1Kgr],23867
6,Hegemonius,stoa014,None,NaN,NaN,None,NaN,NaN,"[Acta Archelai, Acta Archelai]","[1Kgr, 1Kgr]",47325
8,Theocritus,tlg0005,4-3 B.C.,-400.0,-201.0,False,-400.0,-201.0,"[Syrinx, Εἰδύλλια, Ἐπιγράμματα]","[1Kgr, perseus, perseus]",21262
9,Euripides,tlg0006,5 B.C.,-500.0,-401.0,False,-500.0,-401.0,"[Fragmenta, Κύκλωψ, Ἄλκηστις, Μήδεια, Ἡρακλεῖδ...","[1Kgr, perseus, perseus, perseus, perseus, per...",166426
10,Plutarch,tlg0007,A.D. 1-2,1.0,200.0,False,1.0,200.0,"[Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο, Ἐκλογὴ περ...","[1Kgr, 1Kgr, perseus, perseus, perseus, perseu...",1139617
12,Herodian,tlg0015,A.D. 2-3,101.0,300.0,False,101.0,300.0,[Ab excessu divi Marci],[1Kgr],46751


# Merge with our previously coded metadata for authors


In [77]:
authors_old = get_as_dataframe(AGT_overview.worksheet("authors_overview_20201019"))
authors_old.head(5)

,author id,name,provenience,date,N of works,N of words
0,tlg0001,Apollonius Rhodius,pagan,-2.5,1,38822
1,tlg0003,Thucydides,pagan,-4.5,1,150160
2,tlg0004,Diogenes Laertius,pagan,2.5,1,111053
3,tlg0005,Theocritus,pagan,-3.0,2,21321
4,tlg0006,Euripides,pagan,-4.5,13,112900


In [78]:
authors_old.rename(columns={"author id" :  "author_id","name" : "author_man", "date" : "cent_avg_man"}, inplace=True)
authors_old.drop(columns=["N of words", "N of works"], inplace=True)

In [79]:
authors_df = authors_df.merge(authors_old, on="author_id", how="outer")

In [80]:
### some other manual updates....
author_cent_prov = {}
author_cent_prov["tlg0526"] = [1, 100, 'jewish']
author_cent_prov["tlg2040"] = (301, 400, 'christian')
author_cent_prov["tlg1329"] = (101, 200, 'christian')
author_cent_prov["tlg2035"] = (301, 400, 'christian')
author_cent_prov['tlg0563'] = (101, 200, 'christian')
author_cent_prov['tlg0013'] = (-800, -601, 'pagan')
author_cent_prov['tlg2029'] = (201, 400, 'pagan')
author_cent_prov['tlg0656'] = (1, 100, 'pagan')
author_cent_prov['tlg0559'] = (1, 100, 'pagan')
author_cent_prov['tlg0099'] = (-100, 100, 'pagan')

In [82]:
for key, vals in author_cent_prov.items():
  authors_df.loc[authors_df["author_id"] == key, "not_before_man"] = vals[0]
  authors_df.loc[authors_df["author_id"] == key, "not_after_man"] = vals[1]
  authors_df.loc[authors_df["author_id"] == key, "provenience"] = vals[2]

In [83]:
authors_df.columns

Index(['author', 'author_id', 'tlg_date', 'not_before', 'not_after',
       'date_uncertain', 'not_before_man', 'not_after_man', 'titles',
       'sources', 'wordcounts', 'author_man', 'provenience', 'cent_avg_man'],
      dtype='object')

In [85]:
authors_df = authors_df[['author_id', 'author', 'author_man', 'provenience', 'not_before_man', 'not_after_man',  'cent_avg_man', 'titles', 'sources', 'wordcounts']]

In [86]:
set_with_dataframe(AGT_overview.add_worksheet("authors_df_CODING", 1,1), authors_df)

# Add TLG epithets

In [4]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_dated_20201027.json", "df", conf)

In [5]:
resp = requests.get("https://raw.githubusercontent.com/kylepjohnson/cltk/master/cltk/corpus/greek/tlg/author_epithet.json")
tlg_author_epithet = json.loads(resp.content)

tlg_author_epithet_reversed = {}
for key in tlg_author_epithet.keys():
    for author_id in tlg_author_epithet[key]:
        tlg_author_epithet_reversed["tlg" + author_id] = key  
        
tlg_author_epithet_reversed

def get_epithet(author_id): # we use doc_id, since author id was modified
    try:
        epithets = tlg_author_epithet_reversed[author_id[:7]]
    except:
        epithets = []
    return epithets

AGT["tlg_epithet"] = AGT["author_id"].apply(get_epithet)

In [6]:
sddk.write_file("SDAM_data/AGT/AGT_dated_20201126.json", AGT, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/AGT_dated_20201126.json"
